In [1]:
from huggingface_hub import HfApi, notebook_login

# Hugging Face 계정으로 로그인
notebook_login()


In [ ]:
!pip install hf-lfs
!git config --global user.email "endnjs33@gmail.com"
!git config --global user.name "DooWonLee"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 9.8 MB/s eta 0:00:00


In [66]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd

model_name = 'klue/roberta-base'
batch_size = 128
num_labels = 7
task = 'tc'

# 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
# {HuggingFace Model Hub 사용자 아이디}/{push_to_hub_model_id에서 설정한 값}
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# GPU 사용 설정 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

# 데이터 로드 옵션 선택: AWS S3 | Hugging Face

## AWS S3에서 학습 데이터 로드

In [3]:
import os
from dotenv import load_dotenv
from google.colab import drive

drive.mount('/content/drive')

# Google 드라이브 내의 aws.env 파일에서 환경 변수 로드
load_dotenv(dotenv_path='/content/drive/My Drive/Colab_Notebooks/aws.env')

# 환경 변수 사용
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
region = os.getenv('AWS_DEFAULT_REGION')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
!pip install boto3
!pip install python-dotenv
! pip install -U accelerate
! pip install -U transformers
!pip install datasets
!pip install evaluate
!pip install wandb

In [96]:
import boto3
import pandas as pd
from io import StringIO
from sklearn.model_selection import train_test_split
import pandas as pd

# boto3 클라이언트 생성
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region
)

# 파일 로드를 위한 버킷 이름과 파일 경로 설정`
bucket_name = 'ldw-news-train'
file_key = 'training_data_20240304224328.csv'

# S3에서 파일을 문자열로 로드
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
file_content = response['Body'].read().decode('utf-8')

# 문자열에서 DataFrame 생성
df = pd.read_csv(StringIO(file_content))

# DataFrame 출력하여 내용 확인
print(df.head())

# 데이터셋 분할
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# 결측치 확인
print(train_df.isna().sum())
print(test_df.isna().sum())

     id category                            title  \
0  9318     IT과학     네이버, 총선 특별 페이지에 지역 언론사 뉴스 제공   
1  9319     IT과학  네이버 뉴스, 총선 기간 지역뉴스 강화…다양한 정보 제공   
2  9320       사회        유튜브 프리미엄 계정 공유, 싸게 보려다...   
3  9321       정치       임종석 탈락…명문 갈등의 분수령일까? 종식일까?   
4  9322       경제          위생 논란 탄산수 페리에 줄줄이 판매 중단   

                                             content  
0  뉴스혁신포럼 첫 개선 권고책시·도 지역별로 기사 구분해 노출네이버가 내달 오픈 예정...  
1  다음달 네이버 선거 특별 페이지에 지역뉴스 강화네이버는 다음 달 오픈 예정인 제22...  
2  서울시가 최근 유튜브 계정 공유 이용권 관련 피해가 급증하고 있다며 소비자피해주의보...  
3  더불어민주당 공천 갈등 ‘뇌관’ 터졌다… ‘친문 황태자’ 임종석도 탈락끝내 더불어민...  
4  뉴스 속 주인공을 만나보는 ''뉴스메이커''입니다.글로벌 식음료회사 네슬레가 생산해...  
id          0
category    0
title       0
content     4
dtype: int64
id          0
category    0
title       0
content     1
dtype: int64


## Hugging Face 에서 학습 데이터 로드


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset , DatasetDict
from transformers import TrainingArguments, Trainer
from datasets import Dataset, load_dataset
import pandas as pd
import numpy as np

# Hugging Face 데이터셋 로드
dataset = load_dataset("Doowon96/training_data_20240304", split='train')
file_key = "training_data_20240304"
# 데이터셋 분할
split_dataset = dataset.train_test_split(test_size=0.3)

dataset_dict = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

# DataFrame으로 변환
train_df = pd.DataFrame(dataset_dict['train'])
test_df = pd.DataFrame(dataset_dict['test'])

# 결측치 확인
print(train_df.isna().sum())
print(test_df.isna().sum())

id          0
category    0
title       0
content     2
dtype: int64
id          0
category    0
title       0
content     2
dtype: int64


# 모델 파인튜닝

In [97]:
# @title 기본 제목 텍스트 { display-mode: "code" }
from datasets import Dataset, load_dataset
import numpy as np
# Pandas DataFrame을 Hugging Face의 Dataset으로 변환
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

# 카테고리를 정수로 매핑하는 딕셔너리
category_to_id = {
    'IT과학': 0,
    '경제': 1,
    '사회': 2,
    '생활문화': 3,
    '세계': 4,
    '스포츠': 5,
    '정치': 6
}


def preprocess_function(examples):
    # 제목을 토크나이즈
    tokenized_titles = tokenizer(examples['title'], truncation=True, padding=True)

    # 카테고리 레이블을 정수로 변환
    tokenized_titles['labels'] = [category_to_id[category] for category in examples['category']]

    return tokenized_titles

# 전처리 함수 적용
encoded_dataset_train = dataset_train.map(preprocess_function, batched=True)
encoded_dataset_test = dataset_test.map(preprocess_function, batched=True)


Map:   0%|          | 0/1900 [00:00<?, ? examples/s]

Map:   0%|          | 0/815 [00:00<?, ? examples/s]

In [6]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [51]:
model_checkpoint = "klue/roberta-base"

In [9]:
encoded_dataset_train.features['labels']

Value(dtype='int64', id=None)

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
from transformers import TrainingArguments
import os

model_name = model_name.split("/")[-1]
output_dir = os.path.join("test-klue", "ynat")
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch",  # 변경: 에폭 단위로 모델 저장
    load_best_model_at_end=True,
    # save_steps 설정 제거 또는 주석 처리
    # save_steps = 500,  # 주석 처리

    # Dataset
    num_train_epochs=10,  # 훈련 에폭 수
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    # Optimizer
    learning_rate=2e-5,  # 5e-5
    weight_decay=0.01,  # 0
    warmup_steps=10,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,

    # Evaluation
    metric_for_best_model='eval_f1',
    evaluation_strategy = "epoch",  # 변경: 에폭 단위로 평가

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,  # 로그 저장 디렉토리 설정 유지
    report_to='wandb',  # Weights & Biases 로깅 설정 유지

    # Randomness
    seed=42,  # 랜덤 시드 설정 유지
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1834: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Doowon96/roberta-base-finetuned-tc).
  warnings.warn(


In [12]:
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, brier_score, cer, character, charcut_mt, chrf, code_eval, comet, competition_math, confusion_matrix, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, mape, mase, matthews_correlation, mauve, mean_iou, meteor, mse, nist_mt, pearsonr, perplexity, poseval, precision, r_squared, recall, rl_reliability, roc_auc, rouge, sacrebleu, sari, seqeval, smape, spearmanr, squad, squad_v2, super_glue, ter, trec_eval, wer, wiki_split, xnli, xtreme_s, Aledade/extraction_evaluation, AlhitawiMohammed22/CER_Hu-Evaluation-Metrics, Bekhouche/NED, BucketHeadP65/confusion_matrix, BucketHeadP65/roc_curve, CZLC/rouge_raw, DaliaCaRo/accents_unplugged_eval, DarrenChensformer/eval_keyphrase, DarrenChensformer/relation_extraction, DoctorSlimm/bangalore_score, DoctorSlimm/kaushiks_criteria, Drunper/metrica_tesi, Felipehonorato/eer, Fritz02/execution_accuracy, GMFTBY/dailydialog_evaluate, GMFTBY/dailydialogevaluate, He-Xingwei/sari_metric, I

<ipython-input-12-fc3a39300919>:2: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


In [54]:
import evaluate

# 평가 지표 리스트를 로드합니다.
metrics_list = evaluate.list_evaluation_modules()

# 리스트의 길이와 함께 평가 지표를 출력합니다.
print(len(metrics_list))
print(', '.join(metric for metric in metrics_list))


151
lvwerra/test, jordyvl/ece, angelina-wang/directional_bias_amplification, cpllab/syntaxgym, lvwerra/bary_score, hack/test_metric, yzha/ctc_eval, codeparrot/apps_metric, mfumanelli/geometric_mean, daiyizheng/valid, erntkn/dice_coefficient, mgfrantz/roc_auc_macro, Vlasta/pr_auc, gorkaartola/metric_for_tp_fp_samples, idsedykh/metric, idsedykh/codebleu2, idsedykh/codebleu, idsedykh/megaglue, cakiki/ndcg, Vertaix/vendiscore, GMFTBY/dailydialogevaluate, GMFTBY/dailydialog_evaluate, jzm-mailchimp/joshs_second_test_metric, ola13/precision_at_k, yulong-me/yl_metric, abidlabs/mean_iou, abidlabs/mean_iou2, KevinSpaghetti/accuracyk, NimaBoscarino/weat, ronaldahmed/nwentfaithfulness, Viona/infolm, kyokote/my_metric2, kashif/mape, Ochiroo/rouge_mn, giulio98/code_eval_outputs, leslyarun/fbeta_score, giulio98/codebleu, anz2/iliauniiccocrevaluation, zbeloki/m2, xu1998hz/sescore, dvitel/codebleu, NCSOFT/harim_plus, JP-SystemsX/nDCG, sportlosos/sescore, Drunper/metrica_tesi, jpxkqx/peak_signal_to_nois

In [53]:
metric_macrof1 = evaluate.load('f1')

In [59]:
from transformers import DataCollatorWithPadding
import numpy as np
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [98]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [103]:
id = wandb.util.generate_id()
print(id)

8q9mrhvj


In [104]:
wandb.init(project='pipeline',
           entity='wpfkcm33',
           id='8q9mrhvj',
        #    name='[EXPERIMENT_NAME]',
           resume=True,
            )

eval/accuracy,▁▄▆▇▇██████
eval/f1,▁▄▇▇▇██████
eval/loss,█▅▃▂▂▁▁▁▁▁▁
eval/precision,▁▄▆▇▇██████
eval/recall,▁▄▆▇▇██████
eval/runtime,▁▄▅▆▅▆▄▇▅██
eval/samples_per_second,█▅▄▃▄▃▅▁▄▁▁
eval/steps_per_second,█▅▄▃▄▃▅▁▄▁▁
train/epoch,▁▂▃▃▄▅▆▆▇███
train/global_step,▁▂▃▃▄▅▆▆▇███
train/total_flos,▁


In [105]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.359988,0.505521,0.339486,0.255552,0.505521
2,No log,0.960130,0.671166,0.584636,0.604149,0.671166
3,No log,0.738055,0.786503,0.762068,0.743866,0.786503
4,No log,0.640175,0.817178,0.796391,0.779309,0.817178
5,No log,0.588556,0.825767,0.807440,0.816253,0.825767
6,No log,0.571373,0.834356,0.821312,0.828049,0.834356
7,No log,0.561792,0.833129,0.823276,0.838556,0.833129
8,No log,0.555932,0.838037,0.830727,0.842840,0.838037
9,No log,0.551049,0.844172,0.837649,0.846617,0.844172
10,No log,0.554451,0.842945,0.835531,0.845398,0.842945


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint destination directory test-klue/ynat/checkpoint-15 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint destination directory test-klue/ynat/checkpoint-30 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:134

TrainOutput(global_step=150, training_loss=0.6663946024576823, metrics={'train_runtime': 242.2091, 'train_samples_per_second': 78.445, 'train_steps_per_second': 0.619, 'total_flos': 273401104200000.0, 'train_loss': 0.6663946024576823, 'epoch': 10.0})

In [106]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

{'eval_loss': 0.5510489344596863, 'eval_accuracy': 0.8441717791411043, 'eval_f1': 0.8376490560259359, 'eval_precision': 0.8466170589213402, 'eval_recall': 0.8441717791411043, 'eval_runtime': 1.2616, 'eval_samples_per_second': 646.001, 'eval_steps_per_second': 5.548, 'epoch': 10.0}


# 시각화

In [70]:
import matplotlib.pyplot as plt
#정확도(Accuracy)와 손실(Loss) 그래프
def plot_accuracy_loss(training_history):
    epochs = range(1, len(training_history['loss']) + 1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, training_history['accuracy'], label='Training Accuracy')
    plt.plot(epochs, training_history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, training_history['loss'], label='Training Loss')
    plt.plot(epochs, training_history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [71]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
#혼동 행렬(Confusion Matrix)
def plot_confusion_matrix(true_labels, predicted_labels, classes):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()


In [72]:
import numpy as np
#F1 점수, 정밀도, 재현율 바 차트
def plot_performance_metrics(metrics):
    labels = list(metrics.keys())
    values = list(metrics.values())

    index = np.arange(len(labels))
    plt.figure(figsize=(8, 5))
    plt.bar(index, values)
    plt.xlabel('Metrics', fontsize=12)
    plt.ylabel('Value', fontsize=12)
    plt.xticks(index, labels, fontsize=11, rotation=30)
    plt.title('Model Performance Metrics')
    plt.show()


In [73]:
from sklearn.metrics import roc_curve, auc
#ROC 곡선과 AUC
def plot_roc_curve(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()


# 모델 및 성능지표 업로드

In [107]:
import json
import boto3
from datetime import datetime
# 성능 지표를 JSON 파일로 로컬에 저장
date_time_str = file_key.split('_')[-1].split('.')[0]
file_name = f"model_performance_{date_time_str}.json"
with open(file_name, 'w') as f:
    json.dump(evaluation_results, f)

# 로컬에 저장된 JSON 파일을 S3에 업로드
s3_client.upload_file(file_name, bucket_name, f"performance/{file_name}")

In [108]:
#허깅페이스에 모델 업로드
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Doowon96/roberta-base-finetuned-tc/commit/c5d7875611e4f849a3d228b09c68a2e2fff4cc33', commit_message='End of training', commit_description='', oid='c5d7875611e4f849a3d228b09c68a2e2fff4cc33', pr_url=None, pr_revision=None, pr_num=None)

# Gradio 를 통한 모델 서빙

In [ ]:
tokenizer.save_pretrained("./my_model_directory")


('./my_model_directory/tokenizer_config.json',
 './my_model_directory/special_tokens_map.json',
 './my_model_directory/vocab.txt',
 './my_model_directory/added_tokens.json',
 './my_model_directory/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./my_model_directory")

# Hugging Face Hub에 토크나이저 업로드
tokenizer.push_to_hub(repo_id="Doowon96/roberta-base-finetuned-tc",  # "<사용자명>/<저장소명>"
                      organization="Doowon96",  # 선택 사항: 조직명이 있다면 지정 (개인 계정인 경우 생략 가능)
                      use_auth_token=True)  # Hugging Face 인증 토큰 사용

README.md:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Doowon96/roberta-base-finetuned-tc/commit/79d514e3d2f8491f80a8e60f36fff328d6f32d45', commit_message='Upload tokenizer', commit_description='', oid='79d514e3d2f8491f80a8e60f36fff328d6f32d45', pr_url=None, pr_revision=None, pr_num=None)

In [109]:
import gradio as gr
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# 사용할 모델 이름
model_name = "Doowon96/roberta-base-finetuned-tc"

# 가능한 모델 버전 목록 (Commit Hash기반)
model_versions = {
    "roberta0305 버전": "b95cef9256104d6bb6912b9cc9b07822b2fb7df8",
    "roberta0304 버전": "c5d7875611e4f849a3d228b09c68a2e2fff4cc33",
    "roberta0303 버전": "4d5bfd262e10dc6d8edd55b9957023fa5f5186f7",
    "roberta0302 버전": "41fd58c8e31b25b6d18e8cfa30e80aba5dca9c2b",
    "roberta0301 버전": "4f3992e102012b68c44eb89bb7f12eb6d077fcce",
    "roberta0229 버전": "e19197733e10c70dfbb00825481e1fd40e2f2977",
    "roberta0228 버전": "34fe46c31f84d04b1c2da393a0026bd8dcd14b55",
    # 필요한 만큼 추가...
}

# 레이블을 카테고리 이름으로 매핑하는 딕셔너리
label_to_category = {
    'LABEL_0': 'IT과학',
    'LABEL_1': '경제',
    'LABEL_2': '사회',
    'LABEL_3': '생활문화',
    'LABEL_4': '세계',
    'LABEL_5': '스포츠',
    'LABEL_6': '정치'
}
def load_model(version):
    """주어진 버전에 따라 모델을 로드하는 함수"""
    model = AutoModelForSequenceClassification.from_pretrained(model_name, revision=model_versions[version])
    tokenizer = AutoTokenizer.from_pretrained(model_name, revision=model_versions[version])
    return pipeline("text-classification", model=model, tokenizer=tokenizer)

def predict(version, 뉴스제목):
    """사용자가 선택한 버전의 모델로 예측을 수행하는 함수"""
    pipe = load_model(version)
    result = pipe(뉴스제목)[0]
    # 레이블을 카테고리 이름으로 변환
    category_name = label_to_category[result['label']]
    # 변환된 카테고리 이름과 확률을 반환
    return f"카테고리: {category_name}, 확률: {result['score']:.4f}"

iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Radio(list(model_versions.keys()), label="모델 버전 선택"),
        gr.Textbox(lines=2, placeholder="여기에 텍스트를 입력하세요...")
    ],
    outputs="text",
    title="다중 버전 모델 예측",
    description="여러 버전의 모델 중에서 선택하여 예측을 수행합니다."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4c43233ed542de6a16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
